In [77]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split

In [78]:
torch.manual_seed(0)
np.random.seed(0)

In [79]:
df = pd.read_csv("/Users/michaelhein/Programs/Dicebot/outputs/output.txt")

In [80]:
df.head()

,totalDice,player,numPlayerDice,quantityDeclared,diceNumDeclared,1s,2s,3s,4s,5s,6s
0,25,Agg-High,5,9,4,2,0,0,3,0,0
1,24,Agg-Low,5,8,4,0,0,1,2,2,0
2,24,Cond-Low,5,9,2,1,2,1,0,0,1
3,23,Cond-High,5,9,3,1,0,3,0,1,0
4,22,Conserv,4,1,4,0,1,0,2,0,1


In [81]:
df["totalDice"]

0         25
1         24
2         24
3         23
4         22
          ..
590880     6
590881     6
590882     5
590883     4
590884     4
Name: totalDice, Length: 590885, dtype: int64

In [82]:
df.shape

(590885, 11)

In [83]:
df[["totalDice", "player", "numPlayerDice", "quantityDeclared", "diceNumDeclared"]]

,totalDice,player,numPlayerDice,quantityDeclared,diceNumDeclared
0,25,Agg-High,5,9,4
1,24,Agg-Low,5,8,4
2,24,Cond-Low,5,9,2
3,23,Cond-High,5,9,3
4,22,Conserv,4,1,4
...,...,...,...,...,...
590880,6,Agg-High,1,2,5
590881,6,Agg-Low,1,3,4
590882,5,Cond-High,3,2,3
590883,4,Agg-Low,1,1,4


In [84]:
features = df[df.columns[:5]]
features.head()

,totalDice,player,numPlayerDice,quantityDeclared,diceNumDeclared
0,25,Agg-High,5,9,4
1,24,Agg-Low,5,8,4
2,24,Cond-Low,5,9,2
3,23,Cond-High,5,9,3
4,22,Conserv,4,1,4


In [85]:
targets = df[df.columns[5:]]
targets.head()

,1s,2s,3s,4s,5s,6s
0,2,0,0,3,0,0
1,0,0,1,2,2,0
2,1,2,1,0,0,1
3,1,0,3,0,1,0
4,0,1,0,2,0,1


In [86]:
CondLow = df[df["player"] == "Cond-Low"]
CondLow = CondLow.drop(columns = ["player"])
CondLow.head()

,totalDice,numPlayerDice,quantityDeclared,diceNumDeclared,1s,2s,3s,4s,5s,6s
2,24,5,9,2,1,2,1,0,0,1
7,22,4,7,5,0,1,1,0,1,1
12,21,4,8,1,2,0,1,0,1,0
17,19,3,6,4,0,0,0,1,1,1
22,18,3,7,4,0,1,1,1,0,0


In [87]:
td, test = train_test_split(CondLow, test_size=0.1)

In [88]:
train, develop = train_test_split(CondLow, test_size=0.2)

In [89]:
train.shape

(98535, 10)

In [90]:
develop.shape

(24634, 10)

In [91]:
test.shape

(12317, 10)

In [92]:
x_train = train[train.columns[:4]]
y_train = train[train.columns[4:]]

x_develop = develop[develop.columns[:4]]
y_develop = develop[develop.columns[4:]]

x_test = test[test.columns[:4]]
y_test = test[test.columns[4:]]

In [93]:
x_train_tensor = torch.from_numpy(x_train.to_numpy()).float()
y_train_tensor = torch.from_numpy(y_train.to_numpy()).float()

x_develop_tensor = torch.from_numpy(x_develop.to_numpy()).float()
y_develop_tensor = torch.from_numpy(y_develop.to_numpy()).float()

x_test_tensor = torch.from_numpy(x_test.to_numpy()).float()
y_test_tensor = torch.from_numpy(y_test.to_numpy()).float()

In [94]:
train_dataset = list(zip(x_train_tensor, y_train_tensor))
develop_dataset = list(zip(x_develop_tensor, y_develop_tensor))
test_dataset = list(zip(x_test_tensor, y_test_tensor))

In [95]:
class Net(nn.Module):
    def __init__(self, num_layers, layers_size):
        super(Net, self).__init__()
        self.hidden_layers = nn.ModuleList([nn.Linear(4, layers_size)])
        self.hidden_layers.extend([nn.Linear(layers_size, layers_size) for i in range(1, num_layers-1)])
        self.output = nn.Linear(layers_size, 6)
        self.activation = nn.ReLU()
        
    def forward(self, x):
        z = x
        for layer in self.hidden_layers:
            z = self.activation(layer(z))
        return self.output(z)

In [96]:
class Trainer():
    def __init__(self, net, optim, loss_function, train_loader):
        self.net = net
        self.optim = optim
        self.loss_function = loss_function
        self.train_loader = train_loader
        
    def train(self, epochs, verbose=True):
        losses = []
        for epoch in range(epochs):
            epoch_loss = 0.0
            epoch_steps = 0
            for data in self.train_loader:
                x = data[0]
                y = data[1]
                
                self.optim.zero_grad()
                
                preds = self.net.forward(x)
                
                loss = self.loss_function(preds, y)
                
                loss.backward()
                
                self.optim.step()
                
                epoch_loss += loss.item()
                epoch_steps += 1
                
            losses.append(epoch_loss / epoch_steps)
            if verbose: print("epoch [%d]: loss %.3f" % (epoch+1, losses[-1]))
        return losses

In [97]:
## Hyperparamters

num_epochs = 30
batch_size = 256
learning_rate = 0.02

In [98]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
develop_loader = DataLoader(dataset=develop_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [99]:
net = Net(4, 16)
opt = optim.Adam(net.parameters(), lr=learning_rate)
loss_function = nn.MSELoss()

trainer = Trainer(net=net, optim=opt, loss_function=loss_function, train_loader=train_loader)

losses = trainer.train(num_epochs)

epoch [1]: loss 0.325
epoch [2]: loss 0.265
epoch [3]: loss 0.264
epoch [4]: loss 0.261
epoch [5]: loss 0.260
epoch [6]: loss 0.265
epoch [7]: loss 0.260
epoch [8]: loss 0.259
epoch [9]: loss 0.260
epoch [10]: loss 0.258
epoch [11]: loss 0.282
epoch [12]: loss 0.258
epoch [13]: loss 0.257
epoch [14]: loss 0.270
epoch [15]: loss 0.264
epoch [16]: loss 0.257
epoch [17]: loss 0.256
epoch [18]: loss 0.257
epoch [19]: loss 0.259
epoch [20]: loss 0.256
epoch [21]: loss 0.256
epoch [22]: loss 0.256
epoch [23]: loss 0.274
epoch [24]: loss 0.261
epoch [25]: loss 0.269
epoch [26]: loss 0.261
epoch [27]: loss 0.263
epoch [28]: loss 0.261
epoch [29]: loss 0.261
epoch [30]: loss 0.291


In [100]:
CondLow.head()

,totalDice,numPlayerDice,quantityDeclared,diceNumDeclared,1s,2s,3s,4s,5s,6s
2,24,5,9,2,1,2,1,0,0,1
7,22,4,7,5,0,1,1,0,1,1
12,21,4,8,1,2,0,1,0,1,0
17,19,3,6,4,0,0,0,1,1,1
22,18,3,7,4,0,1,1,1,0,0


In [101]:
train_dataset[0]

(tensor([10.,  3.,  3.,  3.]), tensor([0., 0., 2., 1., 0., 0.]))

In [102]:
train_dataset[1]

(tensor([8., 4., 3., 4.]), tensor([0., 1., 1., 2., 0., 0.]))

In [103]:
x_train.head()

,totalDice,numPlayerDice,quantityDeclared,diceNumDeclared
432720,10,3,3,3
399764,8,4,3,4
531537,6,1,1,1
576524,13,3,4,3
101092,21,3,1,4


In [104]:
net.forward(train_dataset[0][0])

tensor([0.3039, 0.2884, 1.3954, 0.2718, 0.2598, 0.4741],
       grad_fn=<AddBackward0>)

In [105]:
net.forward(train_dataset[1][0])

tensor([0.4513, 0.4764, 0.3847, 1.3397, 0.9712, 0.6489],
       grad_fn=<AddBackward0>)

In [106]:
develop_dataset[0]

(tensor([12.,  2.,  2.,  6.]), tensor([0., 0., 0., 0., 0., 2.]))

In [107]:
develop_dataset[1]

(tensor([9., 1., 3., 3.]), tensor([0., 0., 0., 0., 0., 1.]))

In [108]:
develop_dataset[2]

(tensor([16.,  1.,  1.,  5.]), tensor([0., 0., 0., 0., 1., 0.]))

In [109]:
develop_dataset[3]

(tensor([11.,  3.,  4.,  4.]), tensor([0., 0., 1., 1., 1., 0.]))

In [110]:
net.forward(develop_dataset[0][0])

tensor([0.0657, 0.1074, 0.0545, 0.1586, 0.7687, 0.3172],
       grad_fn=<AddBackward0>)

In [111]:
net.forward(develop_dataset[1][0])

tensor([0.0704, 0.1273, 0.9150, 0.0339, 0.0731, 0.1664],
       grad_fn=<AddBackward0>)

In [112]:
net.forward(develop_dataset[2][0])

tensor([0.0657, 0.1074, 0.0545, 0.1586, 0.7687, 0.3172],
       grad_fn=<AddBackward0>)

In [113]:
net.forward(develop_dataset[3][0])

tensor([0.3106, 0.3213, 0.2649, 0.9583, 0.8915, 0.5176],
       grad_fn=<AddBackward0>)

In [114]:
torch.save(net.state_dict(), "/Users/michaelhein/Programs/Dicebot/models/conditional_low_lvl_stat.obj")